In [1]:
import sqlite3
import pandas as pd

# Connexion à la base de données
conn = sqlite3.connect('/content/database/sales.db')

# Charger les données de la table 'sales' dans un DataFrame
data = pd.read_sql_query("SELECT * FROM sales;", conn)

# Afficher les données
print(data.info())
print(data)

# Fermer la connexion
conn.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13115 entries, 0 to 13114
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           13115 non-null  int64  
 1   date         13115 non-null  object 
 2   datetime     13115 non-null  object 
 3   cash_type    13115 non-null  object 
 4   card         13115 non-null  object 
 5   money        13115 non-null  float64
 6   coffee_name  13115 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 717.4+ KB
None
          id        date                 datetime cash_type  \
0          1  2024-03-01  2024-03-01 10:15:50.520      card   
1          2  2024-03-01  2024-03-01 12:19:22.539      card   
2          3  2024-03-01  2024-03-01 12:20:18.089      card   
3          4  2024-03-01  2024-03-01 13:46:33.006      card   
4          5  2024-03-01  2024-03-01 13:48:14.626      card   
...      ...         ...                      ...       ...   
1311

In [26]:
# Conversion des colonnes 'date' et 'datetime' en type datetime
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S.%f')

# Trouver la dernière date dans les données
derniere_date = data['date'].max()
dernier_mois = derniere_date.month
derniere_annee = derniere_date.year

# Déterminer le mois et l'année du mois précédent
if dernier_mois == 1:  # Si janvier, alors le mois précédent est décembre de l'année précédente
    mois_precedent = 12
    annee_precedente = derniere_annee - 1
else:
    mois_precedent = dernier_mois - 1
    annee_precedente = derniere_annee

# Filtrer les données pour le dernier mois
data_dernier_mois = data[(data['date'].dt.month == dernier_mois) & (data['date'].dt.year == derniere_annee)]

# Filtrer les données pour le mois précédent
data_mois_precedent = data[(data['date'].dt.month == mois_precedent) & (data['date'].dt.year == annee_precedente)]

# Calcul des KPIs pour le dernier mois
total_ca_dernier_mois = round(data_dernier_mois['money'].sum(), 0)
nombre_transactions_dernier_mois = len(data_dernier_mois)
moyenne_ca_dernier_mois = round(data_dernier_mois['money'].mean(), 0)
produit_populaire_dernier_mois = data_dernier_mois['coffee_name'].mode()[0]

# Calcul des KPIs pour le mois précédent
total_ca_mois_precedent = round(data_mois_precedent['money'].sum(), 0)
nombre_transactions_mois_precedent = len(data_mois_precedent)
moyenne_ca_mois_precedent = round(data_mois_precedent['money'].mean(), 0)
produit_populaire_mois_precedent = data_mois_precedent['coffee_name'].mode()[0]

# Calcul des pourcentages de variation
pourcentage_variation_ca = ((total_ca_dernier_mois - total_ca_mois_precedent) / total_ca_mois_precedent) * 100 if total_ca_mois_precedent != 0 else 0
pourcentage_variation_transactions = ((nombre_transactions_dernier_mois - nombre_transactions_mois_precedent) / nombre_transactions_mois_precedent) * 100 if nombre_transactions_mois_precedent != 0 else 0
pourcentage_variation_moyenne_ca = ((moyenne_ca_dernier_mois - moyenne_ca_mois_precedent) / moyenne_ca_mois_precedent) * 100 if moyenne_ca_mois_precedent != 0 else 0


# Stocker les résultats dans une variable texte
mois_dernier_nom = derniere_date.strftime("%b %Y")  # Mois abrégé et année
mois_precedent_nom = (derniere_date.replace(day=1) - pd.Timedelta(days=1)).strftime("%b %Y")  # Mois abrégé et année

resultats_kpis = f"""
KPIs pour {mois_dernier_nom} :
- Total du chiffre d'affaires : {total_ca_dernier_mois}
- Nombre de transactions : {nombre_transactions_dernier_mois}
- Moyenne du chiffre d'affaires par transaction : {moyenne_ca_dernier_mois}
- Produit le plus populaire : {produit_populaire_dernier_mois}

KPIs pour {mois_precedent_nom} :
- Total du chiffre d'affaires : {total_ca_mois_precedent}
- Nombre de transactions : {nombre_transactions_mois_precedent}
- Moyenne du chiffre d'affaires par transaction : {moyenne_ca_mois_precedent}
- Produit le plus populaire : {produit_populaire_mois_precedent}

Tendances en pourcentage par rapport à {mois_precedent_nom} :
- Variation du chiffre d'affaires : {pourcentage_variation_ca:.2f}%
- Variation des transactions : {pourcentage_variation_transactions:.2f}%
- Variation de la moyenne du chiffre d'affaires : {pourcentage_variation_moyenne_ca:.2f}%
"""

# Afficher les résultats
print(resultats_kpis)



KPIs pour Dec 2024 :
- Total du chiffre d'affaires : 30265.0
- Nombre de transactions : 945
- Moyenne du chiffre d'affaires par transaction : 32.0
- Produit le plus populaire : Americano with Milk

KPIs pour Nov 2024 :
- Total du chiffre d'affaires : 42953.0
- Nombre de transactions : 1295
- Moyenne du chiffre d'affaires par transaction : 33.0
- Produit le plus populaire : Latte

Tendances en pourcentage par rapport à Nov 2024 :
- Variation du chiffre d'affaires : -29.54%
- Variation des transactions : -27.03%
- Variation de la moyenne du chiffre d'affaires : -3.03%



In [5]:
import warnings

# Ignorer tous les avertissements
warnings.filterwarnings("ignore")

# Lire la clé API depuis le fichier
with open("api/hf_key.txt", "r") as file:
    hf_token = file.read().strip()

# Exemple d'utilisation de la clé
from huggingface_hub import login, whoami

# Authentifiez-vous avec la clé
login(hf_token)

# Vérifier l'authentification
try:
    print(f"Authentification réussie !")
except Exception as e:
    print("Authentification échouée :", str(e))


Authentification réussie !

In [ ]:
!pip install langchain_community

In [20]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=hf_token

In [28]:
from langchain import HuggingFaceHub

# Initialisation du modèle de langage
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0.1, "max_length": 128})

# Création du message basé sur les résultats KPI
question = (
    f"Écrivez un message aux équipes concernant les résultats suivants :\n"
    f"{resultats_kpis}"
)

# Génération de la réponse
response = llm(question)

# Affichage du message généré
print(response)


Écrivez un message aux équipes concernant les résultats suivants :

KPIs pour Dec 2024 :
- Total du chiffre d'affaires : 30265.0
- Nombre de transactions : 945
- Moyenne du chiffre d'affaires par transaction : 32.0
- Produit le plus populaire : Americano with Milk

KPIs pour Nov 2024 :
- Total du chiffre d'affaires : 42953.0
- Nombre de transactions : 1295
- Moyenne du chiffre d'affaires par transaction : 33.0
- Produit le plus populaire : Latte

Tendances en pourcentage par rapport à Nov 2024 :
- Variation du chiffre d'affaires : -29.54%
- Variation des transactions : -27.03%
- Variation de la moyenne du chiffre d'affaires : -3.03%

Bonjour à tous,

Je tiens à vous partager les résultats de notre performance pour le mois de décembre 2024.

Nous avons enregistré un chiffre d'affaires total de 30 265,0 avec un nombre de transactions de 945. La moyenne du chiffre d'affaires par transaction s'élève à 32,0. Le produit le plus populaire ce mois-ci est l'Americano with Milk.

En comparaison 